In [91]:
### Creating Combat Log Table

In [92]:
# what needs to happen is that a table needs to be created that stores each turn in combat and the actions that happened in that turn.
# the table should have the following columns:

# create the table in the form of a dataframe
import pandas as pd
import numpy as np
import sqlite3
    
# create the table
combat_log = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Action Result',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])

In [93]:
import Species
import Backgrounds
import Armor_and_Weapons
import Dice_Rolls
import Establishing_Hierarchy
import CHARACTER_CREATOR
import Monsters

import Character_Actions

In [94]:
def generate_situation(character_num):
    # for the number of characters, randomly generate a Player_Character using Random_Character()
    entities = []
    for i in range(character_num):
        entities.append(CHARACTER_CREATOR.Random_Character())

    # for the number of entities, within each entity's flesh out the Active_Conditions and Circumstances dictionaries
    for entity in range(len(entities)):
        other_entities = entities.copy()
        other_entities.pop(entity)
        
        # I wnant to add a key for each other entity in the Active_Conditions dictionaries
        for other_entity in range(len(other_entities)):
            entities[entity].Active_Conditions[str(entities[other_entity].Name)] = []
        

    return entities

def generate_combat(character_num):
    # create a new combat log
    combat_log_new = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Action Result',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])
    # create situation
    combat_situation = generate_situation(character_num)
    
      # for each entity, create a column for location within a 52x52 grid
    for entity in range(len(combat_situation)):
        combat_situation[entity].Location = [np.random.randint(0,52),np.random.randint(0,52)]        

    # add the Character_Actions.Attack_Action(entity,combat_situation) to each entity's actions
    for entity in range(len(combat_situation)):
        combat_situation[entity].Actions['Attack'] = Character_Actions.Attack_Action(combat_situation[entity],combat_situation,combat_log_new)
        combat_situation[entity].Actions['Dodge'] = Character_Actions.Dodge_Action(combat_situation[entity],combat_situation,combat_log_new)
        
    for entity in range(len(combat_situation)):
        Character_Actions.Equip_Weapon(combat_situation[entity],Character_Actions.Choose_Random_Weapon(combat_situation[entity]))


    # loop that continues combat until conclusion (aka when one creature's current hp is 0 or less)
        # loop that records each round in 
    combat_round = 0
    combat_log_new['Combat Round'] = combat_round
    #while True:
    #    combat_round += 1
    #    combat_log_new['Combat Round'] = combat_round
    
    
    for entity in range(len(combat_situation)):
        # randomly choose between the Attack_Action and the Dodge_Action
        action = np.random.choice(['Attack','Dodge'])
        new_round = combat_situation[entity].Actions[action]
        # concat the dict new_round to the combat_log_new
        combat_log_new = pd.concat([combat_log_new, pd.DataFrame(new_round)], ignore_index=True, axis=0)

    # reorganize the columns in combat_log_new 
    # 'Combat Round' goes first, then any column with 'Acting True' goes next, then the Current_HP columns, then the rest of the columns except for Combat Round,
    combat_log_new = combat_log_new.reindex(columns=['Combat Round'] + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Acting True')]) + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Current_HP')]) + list(combat_log_new.columns[1:][~combat_log_new.columns[1:].str.contains('Acting True') & ~combat_log_new.columns[1:].str.contains('Current_HP')]))
    
    return combat_log_new


In [95]:
# remove the number of columns limit
pd.set_option('display.max_columns', None)

In [96]:
generate_combat(4)

Character_Creator - Random Character Function - Class Choice:  <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Create Character Function - Class: <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Create Character Function - Class: <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Create Character Function - Class: <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Barbarian' from 'c:\\Users\\maxhi\\O

,Combat Round,a1719 Acting True,a7233 Acting True,a2343 Acting True,a7804 Acting True,a1719 Current_HP,a7233 Current_HP,a2343 Current_HP,a7804 Current_HP,Action Number,Action Time,Action Name,Action Type,Target,Action Result,Current Allied Ability Check,Current Allied Attack Roll,Current Allied Saving Throw,Current Allied Damage Roll,Current Enemy Ability Check,Current Enemy Attack Roll,Current Enemy Saving Throw,Current Enemy Damage Roll,a1719 Location,a7233 Location,a2343 Location,a7804 Location,a1719 Temp_HP,a1719 Size,a1719 Walking Speed,a1719 Flying Speed,a1719 Str_Score,a1719 Dex_Score,a1719 Con_Score,a1719 Int_Score,a1719 Wis_Score,a1719 Cha_Score,a1719 Active_Conditions,a1719 Concentrating,a7233 Temp_HP,a7233 Size,a7233 Walking Speed,a7233 Flying Speed,a7233 Str_Score,a7233 Dex_Score,a7233 Con_Score,a7233 Int_Score,a7233 Wis_Score,a7233 Cha_Score,a7233 Active_Conditions,a7233 Concentrating,a2343 Temp_HP,a2343 Size,a2343 Walking Speed,a2343 Flying Speed,a2343 Str_Score,a2343 Dex_Score,a2343 Con_Score,a2343 Int_Score,a2343 Wis_Score,a2343 Cha_Score,a2343 Active_Conditions,a2343 Concentrating,a7804 Temp_HP,a7804 Size,a7804 Walking Speed,a7804 Flying Speed,a7804 Str_Score,a7804 Dex_Score,a7804 Con_Score,a7804 Int_Score,a7804 Wis_Score,a7804 Cha_Score,a7804 Active_Conditions,a7804 Concentrating
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[12, 45]","[8, 0]","[43, 10]","[39, 27]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1.0,0.0,0.0,0.0,11.0,11.0,7.0,12.0,Undetermined,Action,Dodge,Defense,a1719,Dodge,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rolls.Attack_Roll object at 0x000001D1A1...,<Dice_Rolls.Saving_Throw object at 0x000001D1C...,<Dice_Rolls.Damage_Roll object at 0x000001D1C1...,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rolls.Attack_Roll object at 0x000001D1C1...,<Dice_Rolls.Saving_Throw object at 0x000001D1C...,<Dice_Rolls.Damage_Roll object at 0x000001D1C1...,NaN,NaN,NaN,NaN,0.0,Medium,30.0,0.0,8.0,14.0,13.0,10.0,15.0,12.0,NaN,False,0.0,Medium,30.0,0.0,13.0,11.0,14.0,9.0,16.0,15.0,NaN,False,0.0,Medium,30.0,0.0,12.0,10.0,13.0,14.0,15.0,8.0,NaN,False,0.0,Medium,30.0,0.0,8.0,12.0,15.0,13.0,14.0,10.0,NaN,False
2,0,0.0,1.0,0.0,0.0,11.0,11.0,7.0,12.0,Undetermined,Action,Dodge,Defense,a7233,Dodge,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rolls.Attack_Roll object at 0x000001D1A1...,<Dice_Rolls.Saving_Throw object at 0x000001D1C...,<Dice_Rolls.Damage_Roll object at 0x000001D1C1...,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rolls.Attack_Roll object at 0x000001D1C1...,<Dice_Rolls.Saving_Throw object at 0x000001D1C...,<Dice_Rolls.Damage_Roll object at 0x000001D1C1...,NaN,NaN,NaN,NaN,0.0,Medium,30.0,0.0,8.0,14.0,13.0,10.0,15.0,12.0,NaN,False,0.0,Medium,30.0,0.0,13.0,11.0,14.0,9.0,16.0,15.0,NaN,False,0.0,Medium,30.0,0.0,12.0,10.0,13.0,14.0,15.0,8.0,NaN,False,0.0,Medium,30.0,0.0,8.0,12.0,15.0,13.0,14.0,10.0,NaN,False
3,0,0.0,0.0,1.0,0.0,11.0,11.0,7.0,12.0,Undetermined,Action,Dodge,Defense,a2343,Dodge,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rolls.Attack_Roll object at 0x000001D1A1...,<Dice_Rolls.Saving_Throw object at 0x000001D1C...,<Dice_Rolls.Damage_Roll object at 0x000001D1C1...,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rolls.Attack_Roll object at 0x000001D1C1...,<Dice_Rolls.Saving_Throw object at 0x000001D1C...,<Dice_Rolls.Damage_Roll object at 0x000001D1C1...,NaN,NaN,NaN,NaN,0.0,Medium,30.0,0.0,8.0,14.0,13.0,10.0,15.0,12.0,NaN,False,0.0,Medium,30.0,0.0,13.0,11.0,14.0,9.0,16.0,15.0,NaN,False,0.0,Medium,30.0,0.0,12.0,10.0,13.0,14.0,15.0,8.0,NaN,False,0.0,Medium,30.0,0.0,8.0,12.0,15.0,13.0,14.0,10.0,NaN,False
4,0,0.0,0.0,0.0,1.0,11.0,11.0,7.0,12.0,Undetermined,Action,Attack,Offense,a7233,Miss,<Dice_Rolls.Ability_Check object at 0x000001D1...,<Dice_Rol